# Getting RIVM stats on COVID-19 cases in NL and send it via Telegram 

Required libraries:

In [1]:
import pandas as pd
import numpy as np
import cfscrape
from lxml import etree
import io
import re
import requests

Scrapping the RIVM page: 

In [2]:
url="https://www.rivm.nl/coronavirus-kaart-van-nederland-per-gemeente"

header = {'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9',
          'Accept-Encoding': 'gzip, deflate, sdch',
          'Accept-Language' : 'nl-NL,nl;q=0.8,en-US;q=0.6,en;q=0.4',
          'Cache-Control' : 'max-age=0',
          'Connection': 'keep-alive',
          'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/43.0.2357.81 Safari/537.36'}

scraper = cfscrape.create_scraper()
scraped_html=scraper.get(url,headers=header).content
html = etree.HTML(scraped_html)

date = html.xpath("//div[@id='mapTitles']/text()")[0].split('tot en met ')[1].split('"')[0]
date = pd.to_datetime(date, format='%d-%m-%Y').strftime('%d-%m-%Y')
time = html.xpath("//p/text()")[0].split()[5]

data = html.xpath("//div[@id='csvData']/text()")

print("Last update from the RIVM page:",date,time)

Last update from the RIVM page: 24-03-2020 14.00


Loading the data in a dataframe:

In [3]:
df = pd.read_csv(io.StringIO('\n'.join(str(data[0]).split('\n')[1:])), sep=';')

Workaround to get cases in 'unknown' municipalities:

In [4]:
aantal_unknown_gemeente = int(re.findall(r'\d+',html.xpath("//div[@class='container container-spacer-sm content nobg clearfix']/p/text()")[3])[0])
df.loc[0,'Gemeente']='Unknown'
df.loc[0,'Aantal']= aantal_unknown_gemeente

Loading previous day:

In [5]:
previous_day = (pd.to_datetime(date, format='%d-%m-%Y') - np.timedelta64(1, 'D')).strftime('%d-%m-%Y')
df_previous = pd.read_csv("/Users/santannajj/Desktop/covid19_in_nl/data/covid19-nl-"+str(previous_day)+".csv")
print('Previous day:',previous_day)

Previous day: 23-03-2020


Getting the list of requested Gemeentes:

In [6]:
gemeentes_requested=pd.read_csv('/Users/santannajj/Desktop/covid19_in_nl/data/requested_municipalities.csv',header=None)[0].values.tolist()
gemeentes_requested.sort()

Composing the message per Gemeente in the list:

In [7]:
message="COVID19 CASES IN THE NETHERLANDS\n"
message+="HOURLY MESSAGES (From 8-21)!\n"
message+="Source: https://www.rivm.nl/coronavirus-kaart-van-nederland-per-gemeente\n"
message+="RIVM last update: "+str(date)+" "+ str(time)+"\n"

message+="\n[PREVIOUS >> CURRENT NUMBERS]\n"

aantal_total = df['Aantal'].sum()
message+="\n- TOTAL CASES IN NL: " + str(df_previous['Aantal'].sum()) + " >> "+str(aantal_total) +"\n"

for gemeente in gemeentes_requested:
    message += "- Cases in "+gemeente+": "+str(df_previous[df_previous['Gemeente']==gemeente]['Aantal'].values[0])+" >> "+str(df[df['Gemeente']==gemeente]['Aantal'].values[0])+"\n"

# message+="\nNote: RIVM stated that \"the actual number of infections with COVID-19 is higher than the number of reports in this update because not everyone suspected of a COVID-19 infection is tested.\"\n"
message+="\nTELEGRAM FAILS TO SHOW ALL 355 GEMEENTES!\nWhich municipality should I add here?\nYour request will appear in the next hour."
message+="\n\nMore visual information at https://bit.ly/covid19nl"
print(message)          


COVID19 CASES IN THE NETHERLANDS
HOURLY MESSAGES (From 8-21)!
Source: https://www.rivm.nl/coronavirus-kaart-van-nederland-per-gemeente
RIVM last update: 24-03-2020 14.00

[PREVIOUS >> CURRENT NUMBERS]

- TOTAL CASES IN NL: 4749 >> 5560
- Cases in 's-Gravenhage: 53 >> 63
- Cases in 's-Hertogenbosch: 60 >> 67
- Cases in Altena: 25 >> 28
- Cases in Amersfoort: 41 >> 44
- Cases in Amstelveen: 29 >> 33
- Cases in Amsterdam: 203 >> 250
- Cases in Apeldoorn: 26 >> 29
- Cases in Arnhem: 20 >> 23
- Cases in Bergen op Zoom: 12 >> 12
- Cases in Breda: 161 >> 167
- Cases in Doetinchem: 6 >> 8
- Cases in Eindhoven: 55 >> 68
- Cases in Enschede: 20 >> 20
- Cases in Epe: 10 >> 13
- Cases in Haarlem: 29 >> 41
- Cases in Haarlemmermeer: 18 >> 23
- Cases in Hengelo (O): 15 >> 16
- Cases in Houten: 25 >> 29
- Cases in Leiden: 16 >> 20
- Cases in Leusden: 7 >> 9
- Cases in Meppel: 3 >> 3
- Cases in Montferland: 3 >> 3
- Cases in Nieuwegein: 11 >> 11
- Cases in Nijmegen: 79 >> 93
- Cases in Oost Gelre: 2 >

Function to send a Telegram message:

In [8]:
def telegram_bot_sendtext(bot_message,token,chatid):
    bot_token = token
    bot_chatID = chatid
    send_text = 'https://api.telegram.org/bot' + bot_token + '/sendMessage?chat_id=' + bot_chatID + '&parse_mode=Markdown&text=' + bot_message
    response = requests.get(send_text)
    if response.json()['ok']:
        return "Message Sent!"
    else:
        return "Message failed to be sent!"

Instantiating the function and sending a Telegram message:

In [9]:
from bot_credentials import bot_token, bot_chatID
print("bot_token:",bot_token)
print("bot_chatID:",bot_chatID)

In [11]:
telegram_bot_sendtext(message,bot_token,bot_chatID)

'Message Sent!'

Saving the data if it is a new day:

In [15]:
import pathlib
file_name = pathlib.Path('/Users/santannajj/Desktop/covid19_in_nl/data/covid19-nl-'+date+'.csv')
if not file_name.exists ():
    df.to_csv(file_name, index = False)
    print("UHUHHHH new data, bitch!")
else:
    print("We already have this data!")

We already have this data!


Crontab line:
```
0 08-21 * * * /usr/local/bin/python3 /Users/santannajj/Desktop/covid19_in_nl/crawling_stats_from_rivm_covid19.py
```